In [1]:
import sys, os
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import weatherbench2 as wb
import metpy
from metpy.units import units
sys.path.append('/users/mfeldman/code/xarray_parcel/')
import modules.parcel_functions as parcel
parcel.load_moist_adiabat_lookups()
print('modules loaded')

modules loaded


In [13]:
hres='gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr'
era5='gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr'
pangu='gs://weatherbench2/datasets/pangu/2018-2022_0012_0p25.zarr'
datasets=[era5,hres,pangu]

for dataset in datasets[:1]:
    print(dataset)
    xr_dataset=xr.open_zarr(dataset)
#.to(units.mm))
    xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.year.isin([2020]))
    xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.month.isin([7]))#np.arange(3,10)))
    xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.day.isin([1]))
    print(xr_dataset.time)
    plevel=xr_dataset.level
    tlevel=xr_dataset.temperature
    qlevel=xr_dataset.specific_humidity
    dplevel=metpy.calc.dewpoint_from_specific_humidity(plevel* units('hPa'), tlevel* units('K'), qlevel* units('kg/kg'))#.magnitude
    dplevel.data=dplevel.data.to(units.K).magnitude

    psurf=xr_dataset.mean_sea_level_pressure
    tsurf=xr_dataset['2m_temperature']
    qsurf=xr_dataset.specific_humidity.isel(level=-1)
    dpsurf=metpy.calc.dewpoint_from_specific_humidity(psurf* units('hPa'), tsurf* units('K'), qsurf* units('kg/kg'))#.magnitude
    dpsurf.data=dpsurf.data.to(units.K).magnitude
    #dpsurf=dpsurf.magnitude
    #qsurf=xr_dataset.specific_humitidy.interp(level=psurf)

    cape_cin, array = parcel.cape_cin(plevel, tlevel, dplevel, tsurf, psurf,
             dpsurf, vert_dim='level')

gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr
<xarray.DataArray 'time' (time: 4)>
array(['2020-07-01T00:00:00.000000000', '2020-07-01T06:00:00.000000000',
       '2020-07-01T12:00:00.000000000', '2020-07-01T18:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-07-01 ... 2020-07-01T18:00:00


/tmp/ipykernel_2741889/2952647610.py:17: UserWarning: Relative humidity >120%, ensure proper units.
  dplevel=metpy.calc.dewpoint_from_specific_humidity(plevel* units('hPa'), tlevel* units('K'), qlevel* units('kg/kg'))#.magnitude
/tmp/ipykernel_2741889/2952647610.py:23: UserWarning: Relative humidity >120%, ensure proper units.
  dpsurf=metpy.calc.dewpoint_from_specific_humidity(psurf* units('hPa'), tsurf* units('K'), qsurf* units('kg/kg'))#.magnitude


Exception: Cannot infer object returned from running user provided function. Please supply the 'template' kwarg to map_blocks.

In [12]:
    cape_cin, array = parcel.cape_cin(plevel, tlevel, dplevel, tsurf, psurf,
             dpsurf, vert_dim='level')

Exception: Cannot infer object returned from running user provided function. Please supply the 'template' kwarg to map_blocks.

In [9]:
dplevel.data=dplevel.data.to(units.K).magnitude
dplevel.data

dask.array<mul, shape=(13, 4, 721, 1440), dtype=float64, chunksize=(13, 1, 721, 1440), chunktype=numpy.ndarray>